<a href="https://colab.research.google.com/github/andresvc21/Random_Practice_ML/blob/main/Homework_M7_DeepLearning_Andres_Villegas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Homework
Create an RL that can balance pole in the CartPole game for 400 frames.

In [ ]:
!apt update && apt install -y libpq-dev libsdl2-dev swig xorg-dev xvfb
!pip install -q -U tf-agents-nightly pyvirtualdisplay gym[atari]

In [2]:
import gym

from tensorflow import keras
import tensorflow as tf

import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.animation as animation
mpl.rc('animation', html='jshtml')

In [3]:
env = gym.make('CartPole-v0')
env._max_episode_steps = 400

In [4]:
try:
    import pyvirtualdisplay
    display = pyvirtualdisplay.Display(visible=0, size=(1400, 900)).start()
except ImportError:
    pass

In [5]:
env.seed(42)

frames = []
stop = 0

obs = env.reset()

for step in range(400):
    img = env.render(mode="rgb_array")
    
    action = 0 # this is a terrible policy...

    obs, reward, done, info = env.step(action)
    frames.append(img)

    # For the animation we keep printing frames for 50 steps
    stop += done
    if stop > 50:
      break

len(frames)

/usr/local/lib/python3.7/dist-packages/gym/logger.py:30: UserWarning: WARN: You are calling 'step()' even though this environment has already returned done = True. You should always call 'reset()' once you receive 'done = True' -- any further steps are undefined behavior.
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


58

#Answer


Start an instance of the virtual display

In [8]:
from pyvirtualdisplay import Display
display = Display(visible=False, size=(1400, 900))
_ = display.start()

500 steps of recording using videorecorder from OpenAI gym

In [9]:
from gym.wrappers.monitoring.video_recorder import VideoRecorder
before_training = "before_training.mp4"

video = VideoRecorder(env, before_training)
# returns an initial observation
env.reset()
for i in range(500):
  env.render()
  video.capture_frame()
  # env.action_space.sample() produces either 0 (left) or 1 (right).
  observation, reward, done, info = env.step(env.action_space.sample())
  # Not printing this time
  #print("step", i, observation, reward, done, info)

video.close()
env.close()

/usr/local/lib/python3.7/dist-packages/gym/logger.py:30: UserWarning: WARN: You are calling 'step()' even though this environment has already returned done = True. You should always call 'reset()' once you receive 'done = True' -- any further steps are undefined behavior.
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


Function to render the saved video


In [10]:
from base64 import b64encode
def render_mp4(videopath: str) -> str:
  mp4 = open(videopath, 'rb').read()
  base64_encoded_mp4 = b64encode(mp4).decode()
  return f'<video width=400 controls><source src="data:video/mp4;' \
         f'base64,{base64_encoded_mp4}" type="video/mp4"></video>'

In [11]:
from IPython.display import HTML
html = render_mp4(before_training)
HTML(html)

Code from https://www.anyscale.com/blog/an-introduction-to-reinforcement-learning-with-openai-gym-rllib-and-google

In [ ]:
!pip install 'ray[rllib]'==1.6

Usgint Tensorflow using Proximal Policy Optimization (PPO) algorithm. ,

Source: https://docs.ray.io/en/master/rllib/rllib-algorithms.html#proximal-policy-optimization-ppo 

In [ ]:
import ray
from ray.rllib.agents.ppo import PPOTrainer
config = {
    "env": "CartPole-v0",
    "framework": "tf",
    "model": {
      "fcnet_hiddens": [32],
      "fcnet_activation": "linear",
    },
}
stop = {"episode_reward_mean": 195}
ray.shutdown()
ray.init(
num_cpus=3,
include_dashboard=False,
ignore_reinit_error=True,
log_to_driver=False,
)
# execute training 
analysis = ray.tune.run(
  "PPO",
  config=config,
  stop=stop,
  checkpoint_at_end=True,
)

In [15]:
# restore a trainer from the last checkpoint
trial = analysis.get_best_logdir("episode_reward_mean", "max")
checkpoint = analysis.get_best_checkpoint(
  trial,
  "training_iteration",
  "max",
)
trainer = PPOTrainer(config=config)
trainer.restore(checkpoint)

2022-03-06 04:34:16,941	INFO ppo.py:159 -- In multi-agent mode, policies will be optimized sequentially by the multi-GPU optimizer. Consider setting simple_optimizer=True if this doesn't work for you.
2022-03-06 04:34:16,942	INFO trainer.py:728 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.
2022-03-06 04:34:26,777	WARNING util.py:55 -- Install gputil for GPU system monitoring.
2022-03-06 04:34:26,836	INFO trainable.py:383 -- Restored on 172.28.0.2 from checkpoint: /root/ray_results/PPO/PPO_CartPole-v0_63229_00000_0_2022-03-06_04-32-10/checkpoint_000022/checkpoint-22
2022-03-06 04:34:26,838	INFO trainable.py:390 -- Current state after restoring: {'_iteration': 22, '_timesteps_total': None, '_time_total': 104.54067397117615, '_episodes_total': 945}


Render video

In [22]:
after_training = "after_training.mp4"
after_video = VideoRecorder(env, after_training)
observation = env.reset()
done = False

#instead 400 while done

while not done:
  env.render()
  after_video.capture_frame()
  action = trainer.compute_action(observation)
  observation, reward, done, info = env.step(action)
after_video.close()
env.close()

html = render_mp4(after_training)
HTML(html)